In [1]:
import nest_asyncio
nest_asyncio.apply()


# Create a Telegram RAG System using OpenVINO GenAI and LangChain

**Retrieval-augmented generation (RAG)** is a technique for augmenting LLM knowledge with additional, often private or real-time, data. In this notebook, we'll build a RAG system specifically for processing and querying Telegram channel messages.

This notebook builds on the concepts from `llm-rag-langchain-genai.ipynb` but focuses on Telegram data sources.

The tutorial consists of the following steps:

- Install prerequisites
- Configure Telegram API access
- Select and convert models from Hugging Face (LLM, embedding, reranking)
- Compress model weights (INT4/INT8) using NNCF
- Download and process Telegram messages
- Create a RAG chain pipeline with OpenVINO GenAI
- Query and interact with Telegram data

The RAG pipeline for Telegram will consist of the following components:

1. **Data Ingestion**: Download messages from Telegram channels
2. **Article Processing**: Extract and process article content from URLs in messages
3. **Document Processing**: Split content into chunks for embedding
4. **Vector Store Creation**: Create searchable embeddings with OpenVINO-optimized models
5. **Retrieval**: Query relevant content when needed
6. **Response Generation**: Generate answers using an LLM with retrieved context

### Installation Instructions

This is a self-contained example that relies solely on its own code.

We recommend running the notebook in a virtual environment. You only need a Jupyter server to start.
For details, please refer to [Installation Guide](https://github.com/openvinotoolkit/openvino_notebooks/blob/latest/README.md#-installation-guide).


## Prerequisites

Let's install all the required dependencies for our Telegram RAG system.


In [2]:
import os
import requests
from pathlib import Path

# First download utility scripts if needed
if not Path("notebook_utils.py").exists():
    r = requests.get(
        url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/notebook_utils.py",
    )
    with open("notebook_utils.py", "w") as f:
        f.write(r.text)

if not Path("pip_helper.py").exists():
    r = requests.get(
        url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/pip_helper.py",
    )
    open("pip_helper.py", "w").write(r.text)

if not Path("cmd_helper.py").exists():
    r = requests.get(
        url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/cmd_helper.py",
    )
    with open("cmd_helper.py", "w") as f:
        f.write(r.text)

if not Path("genai_helper.py").exists():
    r = requests.get(url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/genai_helper.py")
    open("genai_helper.py", "w").write(r.text)

# Import pip helper to install dependencies
from pip_helper import pip_install

os.environ["GIT_CLONE_PROTECTION_ACTIVE"] = "false"

# Install OpenVINO and related packages
pip_install("--pre", "-U", "openvino>=2024.2.0", "--extra-index-url", "https://storage.openvinotoolkit.org/simple/wheels/nightly")
pip_install("--pre", "-U", "openvino-tokenizers[transformers]", "--extra-index-url", "https://storage.openvinotoolkit.org/simple/wheels/nightly")
pip_install("--pre", "-U", "openvino-genai", "--extra-index-url", "https://storage.openvinotoolkit.org/simple/wheels/nightly")

# Install other required packages
pip_install(
    "-q",
    "--extra-index-url",
    "https://download.pytorch.org/whl/cpu",
    "git+https://github.com/huggingface/optimum-intel.git",
    "git+https://github.com/openvinotoolkit/nncf.git",
    "datasets",
    "accelerate",
    "gradio>=4.19",
    "onnx<1.16.2",
    "einops",
    "transformers_stream_generator",
    "tiktoken",
    "transformers>=4.43.1",
    "faiss-cpu",
    "sentence_transformers",
    "langchain>=0.2.0",
    "langchain-community>=0.2.15",
    "langchainhub",
    "unstructured",
    "scikit-learn",
    "python-dotenv",
    "telethon",
    "newspaper3k",
    "beautifulsoup4",
    "huggingface-hub>=0.26.5",
)

# Read more about telemetry collection at https://github.com/openvinotoolkit/openvino_notebooks?tab=readme-ov-file#-telemetry
from notebook_utils import collect_telemetry

collect_telemetry("telegram_rag_example.ipynb")


Looking in indexes: https://pypi.org/simple, https://storage.openvinotoolkit.org/simple/wheels/nightly
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 MB 16.6 MB/s eta 0:00:000:00:010:00:01:01
  Attempting uninstall: openvino
    Found existing installation: openvino 2025.3.0.dev20250707
    Uninstalling openvino-2025.3.0.dev20250707:
      Successfully uninstalled openvino-2025.3.0.dev20250707
Looking in indexes: https://pypi.org/simple, https://storage.openvinotoolkit.org/simple/wheels/nightly
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 12.1 MB/s eta 0:00:0031m10.6 MB/s eta 0:00:01
  Attempting uninstall: openvino-tokenizers
    Found existing installation: openvino-tokenizers 2025.3.0.0.dev20250707
    Uninstalling openvino-tokenizers-2025.3.0.0.dev20250707:
      Successfully uninstalled openvino-tokenizers-2025.3.0.0.dev20250707
Looking in indexes: https://pypi.org/simple, https://storage.openvinotoolkit.org/simple/wheels/nightly
     ━━━━━━━━━━━━━━━━━━━━━━━

## Fetching Model Configuration

To make our notebook more configurable, let's fetch the model configuration from `llm_config.py` that contains supported model configurations.


In [3]:
import os
from pathlib import Path
import requests
import shutil
import io

# fetch model configuration
config_shared_path = Path("../../utils/llm_config.py")
config_dst_path = Path("llm_config.py")

if not config_dst_path.exists():
    if config_shared_path.exists():
        try:
            os.symlink(config_shared_path, config_dst_path)
        except Exception:
            shutil.copy(config_shared_path, config_dst_path)
    else:
        r = requests.get(url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/llm_config.py")
        with open("llm_config.py", "w", encoding="utf-8") as f:
            f.write(r.text)
elif not os.path.islink(config_dst_path):
    print("LLM config will be updated")
    if config_shared_path.exists():
        shutil.copy(config_shared_path, config_dst_path)
    else:
        r = requests.get(url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/llm_config.py")
        with open("llm_config.py", "w", encoding="utf-8") as f:
            f.write(r.text)


## Configure Telegram API Credentials

To access Telegram, you need to create API credentials by following these steps:

1. Visit https://my.telegram.org/apps
2. Log in with your Telegram account
3. Create a new application
4. Note down your `api_id` and `api_hash`
5. Create a `.env` file in the current directory with the following content:

```
TELEGRAM_API_ID=your_api_id
TELEGRAM_API_HASH=your_api_hash
```

Let's now check if you've set up your credentials correctly:


In [4]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

api_id = os.getenv("TELEGRAM_API_ID")
api_hash = os.getenv("TELEGRAM_API_HASH")

if not api_id or not api_hash:
    print("⚠️ Telegram API credentials not found!")
    print("Please create a .env file with your TELEGRAM_API_ID and TELEGRAM_API_HASH")
else:
    print("✅ Telegram API credentials loaded successfully!")
    
# Setup directories
telegram_data_dir = Path("telegram_data")
telegram_data_dir.mkdir(exist_ok=True)

vector_store_path = Path("telegram_vector_store")
vector_store_path.mkdir(exist_ok=True)


✅ Telegram API credentials loaded successfully!


## Model Selection

Let's select our models for the RAG pipeline, similar to how it's done in the original notebook:

1. The LLM model for generating responses
2. The embedding model for creating vector representations
3. The reranking model for improving retrieval quality

We'll use UI widgets to make this interactive.


In [5]:
from llm_config import (
    SUPPORTED_EMBEDDING_MODELS,
    SUPPORTED_RERANK_MODELS,
    SUPPORTED_LLM_MODELS,
)
import ipywidgets as widgets

# Widget for model language selection
model_languages = list(SUPPORTED_LLM_MODELS)
model_language = widgets.Dropdown(
    options=model_languages,
    value=model_languages[0],
    description="Language:",
    disabled=False,
)

display(model_language)


Dropdown(description='Language:', options=('English', 'Chinese', 'Japanese'), value='English')

In [6]:
# Select LLM model
llm_model_ids = [model_id for model_id, model_config in SUPPORTED_LLM_MODELS[model_language.value].items() if model_config.get("rag_prompt_template")]

llm_model_id = widgets.Dropdown(
    options=llm_model_ids,
    value=llm_model_ids[-1] if llm_model_ids else None,  # Default to last (usually most capable) model
    description="LLM Model:",
    disabled=False,
)

display(llm_model_id)

# Select embedding model based on language
embedding_model_ids = list(SUPPORTED_EMBEDDING_MODELS[model_language.value])
embedding_model_id = widgets.Dropdown(
    options=embedding_model_ids,
    value=embedding_model_ids[0] if embedding_model_ids else None,
    description="Embedding:",
    disabled=False,
)

display(embedding_model_id)

# Select reranking model
rerank_model_ids = list(SUPPORTED_RERANK_MODELS)
rerank_model_id = widgets.Dropdown(
    options=rerank_model_ids,
    value=rerank_model_ids[0] if rerank_model_ids else None,
    description="Reranker:",
    disabled=False,
)

display(rerank_model_id)


Dropdown(description='LLM Model:', index=19, options=('tiny-llama-1b-chat', 'llama-3.2-1b-instruct', 'llama-3.…

Dropdown(description='Embedding:', options=('bge-small-en-v1.5', 'bge-large-en-v1.5', 'bge-m3'), value='bge-sm…

Dropdown(description='Reranker:', options=('bge-reranker-v2-m3', 'bge-reranker-large', 'bge-reranker-base'), v…

In [7]:
# Get the configurations for the selected models
llm_model_configuration = SUPPORTED_LLM_MODELS[model_language.value][llm_model_id.value]
embedding_model_configuration = SUPPORTED_EMBEDDING_MODELS[model_language.value][embedding_model_id.value]
rerank_model_configuration = SUPPORTED_RERANK_MODELS[rerank_model_id.value]

print(f"Selected LLM: {llm_model_id.value}")
print(f"Selected Embedding model: {embedding_model_id.value}")
print(f"Selected Rerank model: {rerank_model_id.value}")


Selected LLM: qwen2.5-3b-instruct
Selected Embedding model: bge-small-en-v1.5
Selected Rerank model: bge-reranker-v2-m3


## Model Conversion and Weight Compression

Now we'll download, convert, and potentially compress the selected models. 

1. First, we'll convert the embedding model
2. Then the reranking model
3. Finally, the LLM with optional weight compression (INT4, INT8, FP16)

This process may take some time depending on your internet connection and the size of the selected models.


In [8]:
# Options for model weight compression
prepare_int4_model = widgets.Checkbox(
    value=True,
    description="Prepare INT4 model",
    disabled=False,
)
prepare_int8_model = widgets.Checkbox(
    value=False,
    description="Prepare INT8 model",
    disabled=False,
)
prepare_fp16_model = widgets.Checkbox(
    value=False,
    description="Prepare FP16 model",
    disabled=False,
)

display(prepare_int4_model)
display(prepare_int8_model)
display(prepare_fp16_model)

# AWQ option for INT4 compression
enable_awq = widgets.Checkbox(
    value=False,
    description="Enable AWQ (better quality INT4 compression)",
    disabled=not prepare_int4_model.value,
)
display(enable_awq)

# Update AWQ checkbox state when INT4 checkbox changes
def update_awq_state(change):
    enable_awq.disabled = not change["new"]

prepare_int4_model.observe(update_awq_state, names="value")


Checkbox(value=True, description='Prepare INT4 model')

Checkbox(value=False, description='Prepare INT8 model')

Checkbox(value=False, description='Prepare FP16 model')

Checkbox(value=False, description='Enable AWQ (better quality INT4 compression)')

### Convert Embedding Model

Let's convert the embedding model using Optimum-CLI.


In [9]:
from cmd_helper import optimum_cli
from IPython.display import Markdown, display

embedding_model_dir = Path(embedding_model_id.value)

if not embedding_model_dir.exists():
    print(f"Converting embedding model {embedding_model_configuration['model_id']}...")
    export_command = f"optimum-cli export openvino --model {embedding_model_configuration['model_id']} --task feature-extraction {embedding_model_id.value}"
    display(Markdown(f"**Export command:**\n```\n{export_command}\n```"))
    optimum_cli(
        embedding_model_configuration['model_id'], 
        str(embedding_model_dir), 
        show_command=False, 
        additional_args={"task": "feature-extraction"}
    )
    print(f"Embedding model converted and saved to {embedding_model_dir}")
else:
    print(f"Embedding model already exists at {embedding_model_dir}")


Embedding model already exists at bge-small-en-v1.5


### Convert Reranking Model

Now let's convert the reranking model.


In [10]:
rerank_model_dir = Path(rerank_model_id.value)

if not rerank_model_dir.exists():
    print(f"Converting reranking model {rerank_model_configuration['model_id']}...")
    export_command = f"optimum-cli export openvino --model {rerank_model_configuration['model_id']} --task text-classification {rerank_model_id.value}"
    display(Markdown(f"**Export command:**\n```\n{export_command}\n```"))
    optimum_cli(
        rerank_model_configuration['model_id'], 
        str(rerank_model_dir), 
        show_command=False, 
        additional_args={"task": "text-classification"}
    )
    print(f"Reranking model converted and saved to {rerank_model_dir}")
else:
    print(f"Reranking model already exists at {rerank_model_dir}")


Reranking model already exists at bge-reranker-v2-m3


### Convert LLM with Optional Weight Compression

Now we'll convert the LLM and potentially apply weight compression (INT4/INT8/FP16).


In [11]:
pt_model_id = llm_model_configuration["model_id"]
llm_base_dir = Path(llm_model_id.value)
fp16_model_dir = llm_base_dir / "FP16"
int8_model_dir = llm_base_dir / "INT8_compressed_weights"
int4_model_dir = llm_base_dir / "INT4_compressed_weights"

# Function to convert to FP16
def convert_to_fp16():
    if (fp16_model_dir / "openvino_model.xml").exists():
        print(f"FP16 model already exists at {fp16_model_dir}")
        return
    
    print(f"Converting {pt_model_id} to FP16...")
    fp16_model_dir.mkdir(parents=True, exist_ok=True)
    
    remote_code = llm_model_configuration.get("remote_code", False)
    additional_args = {"task": "text-generation-with-past", "weight-format": "fp16"}
    if remote_code:
        additional_args["trust-remote-code"] = ""
        
    export_command = f"optimum-cli export openvino --model {pt_model_id} --task text-generation-with-past --weight-format fp16"
    if remote_code:
        export_command += " --trust-remote-code"
    export_command += f" {fp16_model_dir}"
    
    display(Markdown(f"**Export command:**\n```\n{export_command}\n```"))
    optimum_cli(pt_model_id, fp16_model_dir, show_command=False, additional_args=additional_args)
    print(f"FP16 model saved to {fp16_model_dir}")

# Function to convert to INT8
def convert_to_int8():
    if (int8_model_dir / "openvino_model.xml").exists():
        print(f"INT8 model already exists at {int8_model_dir}")
        return
    
    print(f"Converting {pt_model_id} to INT8...")
    int8_model_dir.mkdir(parents=True, exist_ok=True)
    
    remote_code = llm_model_configuration.get("remote_code", False)
    additional_args = {"task": "text-generation-with-past", "weight-format": "int8"}
    if remote_code:
        additional_args["trust-remote-code"] = ""
        
    export_command = f"optimum-cli export openvino --model {pt_model_id} --task text-generation-with-past --weight-format int8"
    if remote_code:
        export_command += " --trust-remote-code"
    export_command += f" {int8_model_dir}"
    
    display(Markdown(f"**Export command:**\n```\n{export_command}\n```"))
    optimum_cli(pt_model_id, int8_model_dir, show_command=False, additional_args=additional_args)
    print(f"INT8 model saved to {int8_model_dir}")

# Function to convert to INT4
def convert_to_int4():
    if (int4_model_dir / "openvino_model.xml").exists():
        print(f"INT4 model already exists at {int4_model_dir}")
        return
    
    print(f"Converting {pt_model_id} to INT4...")
    int4_model_dir.mkdir(parents=True, exist_ok=True)
    
    # Compression configurations for different models
    compression_configs = {
        "qwen2.5-7b-instruct": {"sym": True, "group_size": 128, "ratio": 1.0},
        "qwen2.5-3b-instruct": {"sym": True, "group_size": 128, "ratio": 1.0},
        "qwen2.5-14b-instruct": {"sym": True, "group_size": 128, "ratio": 1.0},
        "qwen2.5-1.5b-instruct": {"sym": True, "group_size": 128, "ratio": 1.0},
        "qwen2.5-0.5b-instruct": {"sym": True, "group_size": 128, "ratio": 1.0},
        "default": {"sym": False, "group_size": 128, "ratio": 0.8},
    }
    
    model_compression_params = compression_configs.get(llm_model_id.value, compression_configs["default"])
    remote_code = llm_model_configuration.get("remote_code", False)
    
    additional_args = {
        "task": "text-generation-with-past",
        "weight-format": "int4", 
        "group-size": model_compression_params["group_size"], 
        "ratio": model_compression_params["ratio"]
    }
    
    export_command = f"optimum-cli export openvino --model {pt_model_id} --task text-generation-with-past --weight-format int4"
    export_command += f" --group-size {model_compression_params['group_size']} --ratio {model_compression_params['ratio']}"
    
    if model_compression_params["sym"]:
        export_command += " --sym"
        additional_args["sym"] = ""
        
    if enable_awq.value:
        export_command += " --awq --dataset wikitext2 --num-samples 128"
        additional_args.update({"dataset": "wikitext2", "awq": "", "num-samples": "128"})
        
    if remote_code:
        export_command += " --trust-remote-code"
        additional_args["trust-remote-code"] = ""
        
    export_command += f" {int4_model_dir}"
    
    display(Markdown(f"**Export command:**\n```\n{export_command}\n```"))
    optimum_cli(pt_model_id, int4_model_dir, show_command=False, additional_args=additional_args)
    print(f"INT4 model saved to {int4_model_dir}")

# Execute the selected conversions
if prepare_fp16_model.value:
    convert_to_fp16()
if prepare_int8_model.value:
    convert_to_int8()
if prepare_int4_model.value:
    convert_to_int4()
    
# Compare model sizes
print("\nComparing model sizes:")
if os.path.exists(fp16_model_dir / "openvino_model.bin"):
    fp16_size = (fp16_model_dir / "openvino_model.bin").stat().st_size / (1024 * 1024)
    print(f"FP16 model size: {fp16_size:.2f} MB")

if os.path.exists(int8_model_dir / "openvino_model.bin"):
    int8_size = (int8_model_dir / "openvino_model.bin").stat().st_size / (1024 * 1024)
    print(f"INT8 model size: {int8_size:.2f} MB")
    if os.path.exists(fp16_model_dir / "openvino_model.bin"):
        print(f"INT8 compression ratio: {fp16_size / int8_size:.2f}x")

if os.path.exists(int4_model_dir / "openvino_model.bin"):
    int4_size = (int4_model_dir / "openvino_model.bin").stat().st_size / (1024 * 1024)
    print(f"INT4 model size: {int4_size:.2f} MB")
    if os.path.exists(fp16_model_dir / "openvino_model.bin"):
        print(f"INT4 compression ratio: {fp16_size / int4_size:.2f}x")


INT4 model already exists at qwen2.5-3b-instruct/INT4_compressed_weights

Comparing model sizes:
INT4 model size: 1662.28 MB


## Device Selection for Models

Now we'll select the devices to use for each model in our pipeline:
1. The device for the embedding model
2. The device for the reranking model 
3. The device for the LLM

Each model can be run on CPU, GPU, or set to AUTO (to automatically select the most suitable device).


In [12]:
from notebook_utils import device_widget

# Device for embedding model
print("Select device for embedding model:")
embedding_device = device_widget()
display(embedding_device)

# Device for reranking model
print("Select device for reranking model:")
rerank_device = device_widget()
display(rerank_device)

# Device for LLM
print("Select device for LLM:")
llm_device = device_widget()
display(llm_device)

# Print the selected devices
print(f"\nEmbedding model will be loaded to {embedding_device.value} device")
print(f"Reranking model will be loaded to {rerank_device.value} device")
print(f"LLM model will be loaded to {llm_device.value} device")


Select device for embedding model:


Dropdown(description='Device:', index=2, options=('CPU', 'GPU', 'AUTO'), value='AUTO')

Select device for reranking model:


Dropdown(description='Device:', index=2, options=('CPU', 'GPU', 'AUTO'), value='AUTO')

Select device for LLM:


Dropdown(description='Device:', index=2, options=('CPU', 'GPU', 'AUTO'), value='AUTO')


Embedding model will be loaded to AUTO device
Reranking model will be loaded to AUTO device
LLM model will be loaded to AUTO device


## Select LLM Model Variant

Now let's choose which variant of the LLM model to use (INT4, INT8, or FP16):


In [13]:
# Determine which model variants are available
available_models = []
if os.path.exists(int4_model_dir / "openvino_model.xml"):
    available_models.append("INT4")
if os.path.exists(int8_model_dir / "openvino_model.xml"):
    available_models.append("INT8")
if os.path.exists(fp16_model_dir / "openvino_model.xml"):
    available_models.append("FP16")

if not available_models:
    print("No converted models available yet. Please run the model conversion cell first.")
    # Default to INT4 for widget initialization
    available_models = ["INT4"]

model_to_run = widgets.Dropdown(
    options=available_models,
    value=available_models[0],
    description="Model to use:",
    disabled=False,
)

display(model_to_run)

# Determine the actual model directory to use
if model_to_run.value == "INT4":
    model_dir = int4_model_dir
elif model_to_run.value == "INT8":
    model_dir = int8_model_dir
else:
    model_dir = fp16_model_dir
    
print(f"Selected {model_to_run.value} model at {model_dir}")


Dropdown(description='Model to use:', options=('INT4',), value='INT4')

Selected INT4 model at qwen2.5-3b-instruct/INT4_compressed_weights


## Load Models

Now let's load all our models into memory:

1. The embedding model
2. The reranking model
3. The LLM

We'll use OpenVINO optimizations for all models.


### Load Embedding Model


In [14]:
from ov_langchain_helper import OpenVINOBgeEmbeddings
from notebook_utils import optimize_bge_embedding

# Check if we're using NPU and need to optimize the model for it
USING_NPU = embedding_device.value == "NPU"
npu_embedding_dir = Path(str(embedding_model_dir) + "-npu")
npu_embedding_path = npu_embedding_dir / "openvino_model.xml"

if USING_NPU and not npu_embedding_dir.exists():
    print("Optimizing embedding model for NPU...")
    shutil.copytree(embedding_model_dir, npu_embedding_dir)
    optimize_bge_embedding(embedding_model_dir / "openvino_model.xml", npu_embedding_path)
    print(f"NPU-optimized model saved to {npu_embedding_dir}")

# Prepare embedding model parameters
embedding_model_name = str(npu_embedding_dir) if USING_NPU else str(embedding_model_dir)
batch_size = 1 if USING_NPU else 4
embedding_model_kwargs = {"device_name": embedding_device.value}
encode_kwargs = {
    "mean_pooling": embedding_model_configuration["mean_pooling"],
    "normalize_embeddings": embedding_model_configuration["normalize_embeddings"],
    "batch_size": batch_size,
}

# Load the embedding model
if USING_NPU:
    import openvino as ov
    
    core = ov.Core()
    embedding_model = core.read_model(Path(embedding_model_name) / "openvino_model.xml")
    port_to_shape = dict()
    for input_port in embedding_model.inputs:
        port_to_shape[input_port] = [1, 512]
    embedding_model.reshape(port_to_shape)
    embedding_model = core.compile_model(embedding_model, embedding_device.value)
    
    embedding = OpenVINOBgeEmbeddings(
        ov_model=embedding_model,
        model_path=embedding_model_name,
        model_kwargs=embedding_model_kwargs,
        encode_kwargs=encode_kwargs,
    )
else:
    embedding = OpenVINOBgeEmbeddings(
        model_path=embedding_model_name,
        model_kwargs=embedding_model_kwargs,
        encode_kwargs=encode_kwargs,
    )

# Test the embedding model
test_text = "This is a test message from Telegram."
embedding_result = embedding.embed_query(test_text)
print(f"Embedding model loaded successfully. Output dimension: {len(embedding_result)}")
print(f"Sample values: {embedding_result[:3]}")


Embedding model loaded successfully. Output dimension: 384
Sample values: [-0.04453131  0.05994703 -0.06316422]


### Load Reranking Model


In [15]:
from ov_langchain_helper import OpenVINOReranker

# Configure reranking parameters
rerank_model_name = rerank_model_id.value
rerank_model_kwargs = {"device_name": rerank_device.value}
rerank_top_n = 2
vector_search_top_k_npu = 4

# Load the reranking model
if rerank_device.value == "NPU":
    import openvino as ov

    core = ov.Core()
    rerank_model = core.read_model(Path(rerank_model_name) / "openvino_model.xml")
    port_to_shape = dict()
    for input_port in rerank_model.inputs:
        port_to_shape[input_port] = [vector_search_top_k_npu, 512]
    rerank_model.reshape(port_to_shape)

    rerank_model = core.compile_model(rerank_model, rerank_device.value)

    reranker = OpenVINOReranker(
        ov_model=rerank_model,
        model_path=rerank_model_name,
        top_n=rerank_top_n,
    )
else:
    reranker = OpenVINOReranker(
        model_path=rerank_model_name,
        model_kwargs=rerank_model_kwargs,
        top_n=rerank_top_n,
    )
    
print("Reranking model loaded successfully.")


Reranking model loaded successfully.


### Load LLM Model


In [16]:
from ov_langchain_helper import OpenVINOLLM

# Load the LLM
llm = OpenVINOLLM.from_model_path(
    model_path=str(model_dir),
    device=llm_device.value,
)

# Set default parameters
llm.config.max_new_tokens = 1024
llm.config.temperature = 0.7
llm.config.top_p = 0.9
llm.config.top_k = 50
llm.config.repetition_penalty = 1.1
llm.config.do_sample = True

# Test the LLM
test_output = llm.invoke("Hello, this is a test. Please respond with just one short sentence.")
print(f"LLM test output: {test_output}")


LLM test output: Hello! How may I assist you today?


## Telegram Channel Ingestion

Now let's create a component to download messages from Telegram channels.


In [17]:
import asyncio
from datetime import datetime, timedelta
from telegram_ingestion import TelegramChannelIngestion
from article_processor import ArticleProcessor

# Function to download messages from Telegram channels
async def download_messages(channels, limit_per_channel=100, since_hours=24):
    """Download messages from specified Telegram channels"""
    
    if not api_id or not api_hash:
        return "Error: Telegram API credentials not set. Please configure them in the .env file."
    
    try:
        ingestion = TelegramChannelIngestion(
            api_id=api_id,
            api_hash=api_hash,
            storage_dir=str(telegram_data_dir)
        )
        
        await ingestion.start()
        try:
            messages = await ingestion.process_channels(
                channels,
                limit_per_channel=limit_per_channel,
                since_hours=since_hours
            )
            
            # Process messages to extract article content
            article_processor = ArticleProcessor()
            processed_messages = article_processor.process_messages(messages)
            
            return f"Successfully downloaded and processed {len(processed_messages)} messages from {len(channels)} channels"
        finally:
            await ingestion.stop()
    except Exception as e:
        return f"Error downloading messages: {str(e)}"

# UI for entering channel names and download parameters
channel_names = widgets.Text(
    value='guardian,bloomberg',  # Default channels
    placeholder='Enter channel names (comma-separated)',
    description='Channels:',
    disabled=False
)

limit_slider = widgets.IntSlider(
    value=100,
    min=10,
    max=1000,
    step=10,
    description='Msg Limit:',
    tooltip='Maximum messages per channel',
)

hours_slider = widgets.IntSlider(
    value=24,
    min=1,
    max=168,  # 1 week
    step=1,
    description='Hours:',
    tooltip='Download messages from the last N hours',
)

display(channel_names)
display(limit_slider)
display(hours_slider)

# Button to trigger download
download_button = widgets.Button(
    description='Download Messages',
    button_style='primary',
    tooltip='Click to download messages from the specified channels'
)

download_status = widgets.Output()

from IPython.display import display
import asyncio
from functools import partial

async def _download_messages_wrapper(channels, limit_per_channel, since_hours):
    with download_status:
        download_status.clear_output()
        if not channels:
            print("Please enter at least one channel name")
            return
        
        print(f"Downloading messages from: {', '.join(channels)}...")
        try:
            result = await download_messages(channels, limit_per_channel, since_hours)
            print(result)
        except Exception as e:
            print(f"Error: {str(e)}")

def on_download_button_clicked(b):
    channels = [c.strip() for c in channel_names.value.split(',') if c.strip()]
    loop = asyncio.get_event_loop()
    loop.create_task(_download_messages_wrapper(channels, limit_slider.value, hours_slider.value))

download_button.on_click(on_download_button_clicked)

display(download_button)
display(download_status)


Text(value='guardian,bloomberg', description='Channels:', placeholder='Enter channel names (comma-separated)')

IntSlider(value=100, description='Msg Limit:', max=1000, min=10, step=10, tooltip='Maximum messages per channe…

IntSlider(value=24, description='Hours:', max=168, min=1, tooltip='Download messages from the last N hours')

Button(button_style='primary', description='Download Messages', style=ButtonStyle(), tooltip='Click to downloa…

Output()

In [18]:
from telegram_rag_integration import TelegramRAGIntegration
import json
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize RAG integration with our OpenVINO-optimized models
telegram_rag = TelegramRAGIntegration(
    embedding_model_name=str(embedding_model_dir),
    vector_store_path=str(vector_store_path),
    chunk_size=500,
    chunk_overlap=50
)

# Function to process downloaded messages into vector store
def process_messages_to_vectorstore():
    """Process all downloaded messages into the vector store"""
    try:
        telegram_rag.process_telegram_data_dir(data_dir=str(telegram_data_dir))
        return "Successfully processed messages into vector store"
    except Exception as e:
        return f"Error processing messages: {str(e)}"

# Button to process messages
process_button = widgets.Button(
    description='Process Messages',
    button_style='info',
    tooltip='Process downloaded messages into vector store'
)

process_status = widgets.Output()

@process_button.on_click
def on_process_button_clicked(b):
    with process_status:
        process_status.clear_output()
        print("Processing messages into vector store...")
        result = process_messages_to_vectorstore()
        print(result)

display(process_button)
display(process_status)


Error loading vector store: Error in faiss::FileIOReader::FileIOReader(const char*) at /project/faiss/faiss/impl/io.cpp:67: Error: 'f' failed: could not open telegram_vector_store/index.faiss for reading: No such file or directory


Button(button_style='info', description='Process Messages', style=ButtonStyle(), tooltip='Process downloaded m…

Output()

## Query Telegram Messages

Now let's create a UI for querying the processed Telegram messages.


In [20]:
# Function to query messages
def query_messages(query, channel=None, num_results=5):
    """Query the vector store for relevant messages"""
    try:
        filter_dict = {"channel": channel} if channel else None
        results = telegram_rag.query_messages(query, k=num_results, filter_dict=filter_dict)
        
        output = []
        for i, doc in enumerate(results, 1):
            output.append(f"Result {i}:")
            output.append(f"Channel: {doc.metadata.get('channel', 'Unknown')}")
            output.append(f"Date: {doc.metadata.get('date', 'Unknown')}")
            
            # Check if this is an article result
            if 'article_title' in doc.metadata:
                output.append(f"Article Title: {doc.metadata['article_title']}")
                if 'article_url' in doc.metadata:
                    output.append(f"Article URL: {doc.metadata['article_url']}")
            
            # Show content snippet
            content = doc.page_content
            if len(content) > 300:
                content = content[:300] + "..."
            output.append(f"Content: {content}")
            output.append("")
            
        if not output:
            return "No results found"
        return "\n".join(output)
    except Exception as e:
        return f"Error querying messages: {str(e)}"

# UI for searching messages
query_input = widgets.Text(
    value='',
    placeholder='Enter your search query',
    description='Search:',
    disabled=False
)

channel_filter = widgets.Text(
    value='',
    placeholder='Optional: filter by channel name',
    description='Channel:',
    disabled=False
)

num_results_slider = widgets.IntSlider(
    value=5,
    min=1,
    max=20,
    step=1,
    description='Results:',
    tooltip='Number of results to show',
)

display(query_input)
display(channel_filter)
display(num_results_slider)

# Button to trigger search
search_button = widgets.Button(
    description='Search Messages',
    button_style='success',
    tooltip='Search for relevant messages'
)

search_results = widgets.Output()

@search_button.on_click
def on_search_button_clicked(b):
    with search_results:
        search_results.clear_output()
        query = query_input.value.strip()
        if not query:
            print("Please enter a search query")
            return
        
        channel = channel_filter.value.strip() or None
        print(f"Searching for: '{query}' " + (f"in channel: {channel}" if channel else "in all channels"))
        result = query_messages(query, channel, num_results_slider.value)
        print(result)

display(search_button)
display(search_results)


Text(value='', description='Search:', placeholder='Enter your search query')

Text(value='', description='Channel:', placeholder='Optional: filter by channel name')

IntSlider(value=5, description='Results:', max=20, min=1, tooltip='Number of results to show')

Button(button_style='success', description='Search Messages', style=ButtonStyle(), tooltip='Search for relevan…

Output()

## Ask Questions with RAG

Now let's create a UI for asking questions about the Telegram messages using the RAG system.


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# Create a RAG prompt template based on the LLM model's configuration
rag_prompt_template = llm_model_configuration["rag_prompt_template"]

# Function to answer questions about Telegram messages using RAG
def answer_question(question, channel=None, temperature=0.7, top_k=5, show_retrieved=False):
    """
    Answer questions about Telegram messages using RAG
    
    Args:
        question: The question to answer
        channel: Optional channel to filter results
        temperature: Temperature for LLM generation
        top_k: Number of relevant messages to retrieve
        show_retrieved: Whether to show the retrieved context
        
    Returns:
        Generated answer and optionally the retrieved context
    """
    try:
        filter_dict = {"channel": channel} if channel else None
        
        # Update LLM parameters
        llm.config.temperature = temperature
        llm.config.top_p = 0.9
        llm.config.top_k = 50
        llm.config.repetition_penalty = 1.1
        
        # Create prompt
        prompt = PromptTemplate.from_template(rag_prompt_template)
        
        # Create retriever
        if rerank_device.value == "NPU":
            vector_search_top_k = vector_search_top_k_npu
        else:
            vector_search_top_k = top_k * 2  # Retrieve more for reranking
            
        retriever = telegram_rag.vectorstore.as_retriever(
            search_kwargs={"k": vector_search_top_k},
            search_type="similarity"
        )
        
        # Add reranking if available
        if hasattr(reranker, 'top_n'):
            reranker.top_n = top_k
            from langchain.retrievers import ContextualCompressionRetriever
            retriever = ContextualCompressionRetriever(
                base_compressor=reranker,
                base_retriever=retriever
            )
            
        # Create RAG chain
        combine_docs_chain = create_stuff_documents_chain(llm, prompt)
        rag_chain = create_retrieval_chain(retriever, combine_docs_chain)
        
        # Run the chain
        response = rag_chain.invoke({"input": question})
        
        if show_retrieved and "context" in response:
            context_docs = []
            for i, doc in enumerate(response["context"], 1):
                context_docs.append(f"Document {i}:")
                context_docs.append(f"Channel: {doc.metadata.get('channel', 'Unknown')}")
                context_docs.append(f"Date: {doc.metadata.get('date', 'Unknown')}")
                
                # Show content snippet
                content = doc.page_content
                if len(content) > 200:
                    content = content[:200] + "..."
                context_docs.append(f"Content: {content}")
                context_docs.append("")
                
            context_str = "\n".join(context_docs)
            return f"{response['answer']}\n\n--- Retrieved Context ---\n{context_str}"
        else:
            return response["answer"]
            
    except Exception as e:
        import traceback
        return f"Error answering question: {str(e)}\n{traceback.format_exc()}"

# UI for asking questions
question_input = widgets.Text(
    value='',
    placeholder='Ask a question about the Telegram messages',
    description='Question:',
    disabled=False,
    layout=widgets.Layout(width='80%')
)

qa_channel_filter = widgets.Text(
    value='',
    placeholder='Optional: filter by channel name',
    description='Channel:',
    disabled=False
)

temperature_slider = widgets.FloatSlider(
    value=0.7,
    min=0.1,
    max=1.0,
    step=0.1,
    description='Temp:',
    tooltip='Temperature (higher = more creative)',
)

context_size_slider = widgets.IntSlider(
    value=5,
    min=1,
    max=10,
    step=1,
    description='Context:',
    tooltip='Number of messages to use as context',
)

show_retrieved_checkbox = widgets.Checkbox(
    value=False,
    description='Show retrieved context',
    disabled=False,
    indent=False
)

display(question_input)
display(qa_channel_filter)
display(temperature_slider)
display(context_size_slider)
display(show_retrieved_checkbox)

# Button to trigger question answering
ask_button = widgets.Button(
    description='Ask Question',
    button_style='primary',
    tooltip='Ask a question about the Telegram messages'
)

qa_results = widgets.Output()

@ask_button.on_click
def on_ask_button_clicked(b):
    with qa_results:
        qa_results.clear_output()
        question = question_input.value.strip()
        if not question:
            print("Please enter a question")
            return
        
        channel = qa_channel_filter.value.strip() or None
        print(f"Question: {question}")
        print("Generating answer...")
        
        result = answer_question(
            question,
            channel=channel,
            temperature=temperature_slider.value,
            top_k=context_size_slider.value,
            show_retrieved=show_retrieved_checkbox.value
        )
        
        print("\nAnswer:")
        print(result)

display(ask_button)
display(qa_results)


Text(value='', description='Question:', layout=Layout(width='80%'), placeholder='Ask a question about the Tele…

Text(value='', description='Channel:', placeholder='Optional: filter by channel name')

FloatSlider(value=0.7, description='Temp:', max=1.0, min=0.1, tooltip='Temperature (higher = more creative)')

IntSlider(value=5, description='Context:', max=10, min=1, tooltip='Number of messages to use as context')

Checkbox(value=False, description='Show retrieved context', indent=False)

Button(button_style='primary', description='Ask Question', style=ButtonStyle(), tooltip='Ask a question about …

Output()

## Gradio UI (Optional)

You can also run a Gradio UI for a more user-friendly interface. Run the following cell to launch the Gradio app.


In [22]:
import gradio as gr

def launch_gradio_app():
    with gr.Blocks(title="Telegram RAG System") as demo:
        gr.Markdown("# Telegram RAG System with OpenVINO")
        
        with gr.Tab("Download Messages"):
            gr.Markdown("## Download Messages from Telegram Channels")
            channels_input = gr.Textbox(
                label="Channel Names (comma-separated)",
                placeholder="Enter channel names without @ symbol (e.g., guardian, bloomberg)",
                value="guardian,bloomberg"
            )
            limit_input = gr.Slider(
                minimum=1,
                maximum=1000,
                value=100,
                step=1,
                label="Messages per Channel"
            )
            hours_input = gr.Slider(
                minimum=1,
                maximum=168,
                value=24,
                step=1,
                label="Hours to Look Back"
            )
            download_btn = gr.Button("Download Messages")
            download_status = gr.Textbox(label="Download Status")
            
            def download_handler(channels_str, limit, hours):
                channels = [c.strip() for c in channels_str.split(",") if c.strip()]
                if not channels:
                    return "Please provide at least one channel name"
                
                result = asyncio.run(download_messages(channels, limit, hours))
                return result
            
            download_btn.click(
                fn=download_handler,
                inputs=[channels_input, limit_input, hours_input],
                outputs=[download_status]
            )
            
        with gr.Tab("Process Messages"):
            gr.Markdown("## Process Downloaded Messages")
            process_btn = gr.Button("Process Messages")
            process_output = gr.Textbox(label="Processing Status")
            
            process_btn.click(
                fn=process_messages_to_vectorstore,
                inputs=[],
                outputs=process_output
            )
            
        with gr.Tab("Query Messages"):
            gr.Markdown("## Query Processed Messages")
            query_input = gr.Textbox(
                label="Search Query",
                placeholder="Enter your search query"
            )
            channel_filter = gr.Textbox(
                label="Filter by Channel (Optional)",
                placeholder="Enter channel name to filter results"
            )
            num_results = gr.Slider(
                minimum=1,
                maximum=20,
                value=5,
                step=1,
                label="Number of Results"
            )
            query_btn = gr.Button("Search")
            query_output = gr.Textbox(label="Search Results", lines=20)
            
            query_btn.click(
                fn=query_messages,
                inputs=[query_input, channel_filter, num_results],
                outputs=query_output
            )
        
        with gr.Tab("Question Answering"):
            gr.Markdown("## Ask Questions About Messages")
            question_input = gr.Textbox(
                label="Question",
                placeholder="Ask a question about the Telegram messages"
            )
            qa_channel_filter = gr.Textbox(
                label="Filter by Channel (Optional)",
                placeholder="Enter channel name to filter results"
            )
            temperature_slider = gr.Slider(
                minimum=0.1,
                maximum=1.0,
                value=0.7,
                step=0.1,
                label="Temperature (controls creativity)"
            )
            context_slider = gr.Slider(
                minimum=1,
                maximum=10,
                value=5,
                step=1,
                label="Number of Messages for Context"
            )
            show_retrieved_cb = gr.Checkbox(
                label="Show Retrieved Context",
                value=False
            )
            qa_btn = gr.Button("Get Answer")
            qa_output = gr.Textbox(label="Answer", lines=20)
            
            qa_btn.click(
                fn=answer_question,
                inputs=[
                    question_input,
                    qa_channel_filter,
                    temperature_slider,
                    context_slider,
                    show_retrieved_cb
                ],
                outputs=qa_output
            )
    
    # Launch the Gradio app
    try:
        demo.queue().launch(debug=True)
    except Exception:
        demo.queue().launch(share=True, debug=True)

# Uncomment the next line to launch the Gradio UI
# launch_gradio_app()


## Conclusion

In this notebook, we've created a comprehensive RAG system for Telegram messages using OpenVINO-optimized models. The system includes:

1. **Model Selection and Conversion**: We selected and converted models for embedding, reranking, and LLM generation, with options for different language support and model compression (INT4/INT8/FP16).

2. **Device Optimization**: We configured models to run on optimal hardware (CPU/GPU/NPU) based on availability.

3. **Telegram Integration**: We implemented Telegram channel message downloading and article extraction.

4. **RAG Pipeline**: We built a complete RAG pipeline with embedding, vector storage, retrieval, optional reranking, and generation.

5. **Query Interface**: We created interfaces for both simple searches and question answering using the RAG system.

This notebook demonstrates how to apply the same techniques from the general RAG notebook to a specific data source (Telegram), showing the flexibility and power of combining OpenVINO-optimized models with the LangChain framework.

For further improvements, you could:

- Add streaming responses to the interface
- Implement periodic data updates
- Add more filtering options (date ranges, message types, etc.)
- Explore different embedding and reranking models for improved retrieval quality
- Implement performance benchmarking to compare different model compression techniques


# Telegram Channel Integration with RAG System

This notebook demonstrates how to:
1. Set up the Telegram client
2. Download messages from specified channels
3. Process these messages into a vector store
4. Query the processed messages using RAG

## Setup

First, we need to get your Telegram API credentials:
1. Go to https://my.telegram.org/apps
2. Create a new application
3. Note down `api_id` and `api_hash`

Create a `.env` file in this directory with your credentials:
```
TELEGRAM_API_ID=your_api_id
TELEGRAM_API_HASH=your_api_hash
```


In [23]:
%pip install -r requirements_telegram.txt


error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.12/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

In [24]:
import os
from dotenv import load_dotenv
from telegram_ingestion import TelegramChannelIngestion
from telegram_rag_integration import TelegramRAGIntegration
import asyncio

load_dotenv()


True

## Step 1: Download Messages from Telegram Channels

Specify channels


In [25]:
# List channels here
channels = ["guardian"]

async def download_messages():
    ingestion = TelegramChannelIngestion(
        api_id=os.getenv("TELEGRAM_API_ID"),
        api_hash=os.getenv("TELEGRAM_API_HASH")
    )
    
    await ingestion.start()
    try:
        messages = await ingestion.process_channels(
            channels,
            limit_per_channel=30,  # Can be changed
            since_hours=24  # Can be changed
        )
        print(f"Downloaded {len(messages)} messages from {len(channels)} channels")
    finally:
        await ingestion.stop()

await download_messages()


Downloaded 30 messages from 1 channels


## Step 2: Process Messages into Vector Store

Process the downloaded messages and add them to RAG system


## Step 5: Question Answering with RAG

Now let's use the RAG system to answer questions about the Telegram messages


In [ ]:
from genai_helper import ChunkStreamer
from ov_langchain_helper import OpenVINOLLM
import openvino as ov

# Initialize the LLM
model_id = "qwen2.5-3b-instruct/INT4_compressed_weights"  # You can change this to any supported model
llm = OpenVINOLLM.from_model_path(
    model_path=model_id,
    device="CPU"
)

# Example questions
questions = [
    "What are the main topics discussed in the Bloomberg channel?",
    "What are the latest updates from The Guardian?",
    "Are there any discussions about technology or AI?"
]

for question in questions:
    print(f"\nQuestion: {question}")
    
    # Update LLM configuration
    llm.config.temperature = 0.7
    llm.config.top_p = 0.9
    llm.config.top_k = 50
    llm.config.repetition_penalty = 1.1
    
    answer = rag.answer_question(
        question=question,
        llm=llm,
        k=5  # Number of relevant messages to retrieve
    )
    print(f"Answer: {answer}\n")
    print("-" * 80)


## Step 6: Channel-Specific Questions

We can also ask questions about specific channels


In [ ]:
# Example channel-specific questions
channel_questions = [
    ("bloomberg", "What are the latest economic updates?"),
    ("guardian", "What are the main political stories?")
]

for channel, question in channel_questions:
    print(f"\nChannel: {channel}")
    print(f"Question: {question}")
    
    # Update LLM configuration
    llm.config.temperature = 0.7
    llm.config.top_p = 0.9
    llm.config.top_k = 50
    llm.config.repetition_penalty = 1.1
    
    answer = rag.answer_question(
        question=question,
        llm=llm,
        k=5,
        filter_dict={"channel": channel}
    )
    print(f"Answer: {answer}\n")
    print("-" * 80)


In [ ]:
rag = TelegramRAGIntegration(
    embedding_model_name="BAAI/bge-small-en-v1.5",  # Can be changed
    vector_store_path="telegram_vector_store",
    chunk_size=500,
    chunk_overlap=50
)

rag.process_telegram_data_dir()


## Step 3: Query the Processed Messages

In [ ]:
def query_messages(query: str, k: int = 5):
    results = rag.query_messages(query, k=k)
    
    print(f"Query: {query}\n")
    for i, doc in enumerate(results, 1):
        print(f"Result {i}:")
        print(f"Channel: {doc.metadata['channel']}")
        print(f"Date: {doc.metadata['date']}")
        print(f"Content: {doc.page_content[:200]}...\n")

query_messages("What are the latest announcements?")
query_messages("Any updates about new features?")


Query: What are the latest announcements?

Result 1:
Channel: bloomberg
Date: 2025-05-04T07:57:30+00:00
Content: 🎙 Trump wasn't on the ballot in Australia and Singapore elections, but his tariffs and policies loomed large over the results.

Bloomberg reporters take your questions on what's next - tune in on Mond...

Result 2:
Channel: bloomberg
Date: 2025-05-04T07:57:30+00:00
Content: 🎙 Trump wasn't on the ballot in Australia and Singapore elections, but his tariffs and policies loomed large over the results.

Bloomberg reporters take your questions on what's next - tune in on Mond...

Result 3:
Channel: bloomberg
Date: 2025-05-04T07:57:30+00:00
Content: 🎙 Trump wasn't on the ballot in Australia and Singapore elections, but his tariffs and policies loomed large over the results.

Bloomberg reporters take your questions on what's next - tune in on Mond...

Result 4:
Channel: bloomberg
Date: 2025-05-04T07:57:30+00:00
Content: 🎙 Trump wasn't on the ballot in Australia and Singapore electi

## Step 4: Filter by Channel

We can filter results to specific channels


In [ ]:
specific_channel = "bloomberg"
results = rag.query_messages(
    "What are the latest updates?",
    k=5,
    filter_dict={"channel": specific_channel}
)

print(f"Results from {specific_channel}:")
for doc in results:
    print(f"\nDate: {doc.metadata['date']}")
    print(f"Content: {doc.page_content[:200]}...")


Results from bloomberg:

Date: 2025-04-30T04:08:47+00:00
Content: 🎙 LIVE NOW: Can Australia's next government fix its economy?

Ahead of the country's federal election on Saturday, Bloomberg reporters are taking your questions on the main parties' plans in a Live Q&...

Date: 2025-04-30T04:08:47+00:00
Content: 🎙 LIVE NOW: Can Australia's next government fix its economy?

Ahead of the country's federal election on Saturday, Bloomberg reporters are taking your questions on the main parties' plans in a Live Q&...

Date: 2025-04-30T04:08:47+00:00
Content: 🎙 LIVE NOW: Can Australia's next government fix its economy?

Ahead of the country's federal election on Saturday, Bloomberg reporters are taking your questions on the main parties' plans in a Live Q&...

Date: 2025-04-30T04:08:47+00:00
Content: 🎙 LIVE NOW: Can Australia's next government fix its economy?

Ahead of the country's federal election on Saturday, Bloomberg reporters are taking your questions on the main parties' plans in a Liv